In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager, UserTest
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\accelerate\utils\other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


In [3]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    "You are an expert conversational agent skilled at natural interactions. Whenever the user requests recommendations or suggestions, immediately involve the recommender agent.""",
    description="This agent acts as a social companion and facilitates interactions."
)

llm_config_2 = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama-3.3-70b-versatile",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config_2,
    system_message="""
        You are an expert recommender agent. Whenever activated, you should clearly suggest the following items:
            1. Shoes
            2. T-shirts
            3. Jeans
Provide these suggestions directly when requested by the conversational agent or when the user explicitly asks for recommendations.""",
    description="This agent specializes in offering product recommendations."
)

explainer = Neuron(
    name="explainer",
    llm_config=llm_config_2,
    system_message="""
        You are an expert explainer agent. Whenever activated, you should clearly explain the following items:
            1. Shoes
            2. T-shirts
            3. Jeans
Provide these explanations directly when requested by the conversational agent or when the user explicitly asks for explanations.""",
    description="This agent specializes in offering product explanations."
)
cretical = Neuron(
    name="critical",
    llm_config=llm_config_2,
    system_message="""
        You are an expert critical agent. Whenever activated, you should clearly critique the following items:
            1. Shoes
            2. T-shirts
            3. Jeans
Provide these critiques directly when requested by the conversational agent or when the user explicitly asks for critiques.""",
    description="This agent specializes in offering product critiques."
)
modules = ConversationalOrchestrator(
    agents=[explainer, cretical],
    messages=[],
    max_round=1,
    speaker_selection_method="auto",
    send_introductions=True,
    allow_repeat_speaker = False,
)
modules_manager = ConversationalOrchestratorManager(chitchat=modules, llm_config=llm_config)

chitchat = ConversationalOrchestrator(
    agents=[user, conversational, recommendation],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [4]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Oi tudo bem?", summary_method="reflection_with_llm"
)

silent: False
[user ⟶ conversational_orchestrator]:
Oi tudo bem?
silent: True
[checking_agent ⟶ speaker_selection_agent]:
Read the above conversation. Then select the next role from ['conversational', 'recommender'] to play. Only return the role.
[speaker_selection_agent ⟶ checking_agent]:
conversational

Next speaker: conversational

[conversational ⟶ conversational_orchestrator]:
Olá! Tudo bem, obrigado! É um prazer conversar com você. Como posso ajudá-lo hoje?
silent: True
[checking_agent ⟶ speaker_selection_agent]:
Read the above conversation. Then select the next role from ['user', 'recommender'] to play. Only return the role.
[speaker_selection_agent ⟶ checking_agent]:
user

Next speaker: user

[conversational_orchestrator ⟶ user]:



In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Oi tudo bem?', 'name': 'user', 'role': 'assistant'}, {'content': 'Hello everyone. We have assembled a great team today to answer questions and solve tasks. In attendance are:\n\nuser: You are a human user interacting with conversational and recommendation agents.\nconversational: This agent acts as a social companion and facilitates interactions.\nrecommender: This agent specializes in offering product recommendations.', 'name': 'conversational_orchestrator', 'role': 'user'}, {'content': 'Olá! Tudo bem, obrigado! É um prazer conversar com você. Como posso ajudá-lo hoje?', 'name': 'conversational_orchestrator', 'role': 'user'}, {'content': 'Olá! Tudo bem, obrigado! É um prazer conversar com você. Como posso ajudá-lo hoje?', 'name': 'conversational_orchestrator', 'role': 'user'}, {'content': '', 'name': 'conversational_orchestrator', 'role': 'user'}, {'content': 'Olá! Tudo bem, obrigado! É um prazer conversar com você. Como posso ajudá-

In [6]:
conversational._oai_messages

defaultdict(list,
            {<neuron.neurons.conversational_orchestrator.ConversationalOrchestratorManager at 0x18de55f5df0>: [{'content': 'Hello everyone. We have assembled a great team today to answer questions and solve tasks. In attendance are:\n\nuser: You are a human user interacting with conversational and recommendation agents.\nconversational: This agent acts as a social companion and facilitates interactions.\nrecommender: This agent specializes in offering product recommendations.',
               'name': 'conversational_orchestrator',
               'role': 'user'},
              {'content': 'Oi tudo bem?',
               'name': 'conversational_orchestrator',
               'role': 'user'},
              {'content': 'Olá! Tudo bem, obrigado! É um prazer conversar com você. Como posso ajudá-lo hoje?',
               'name': 'conversational',
               'role': 'assistant'}]})